In [2]:
import pandas as pd
from pymongo import MongoClient
from recommender.infrastructure.repository.mongodb import (
    MongoDBTracksRepository
)


Open connection to DB

In [3]:
client = MongoClient()
db = client.mgr
plays_repository = MongoDBTracksRepository(db.playedtracks)

Get all rows from DB, and create a dataframe with all the song plays

In [30]:
records = plays_repository.all_raw()
plays = pd.DataFrame(list(records))
plays

,_id,artist,name,tags,loved,user_playcount,total_playcount,playback_utc_date,mbid
0,615da9a3690c7486296ebdb9,"{'name': 'The Crystal Method', 'mbid': 'aaf09f...",Name of the Game (edit),"[electronic, electronica, the crystal method, ...",False,0,20575,2007-07-31 01:12:10,None
1,615da9a0690c7486296ebdb8,"{'name': 'Tiësto', 'mbid': 'aabb1d9f-be12-45b3...",Sweet Things,"[trance, Tiesto, electronic, vocal trance, ele...",False,0,131760,2007-07-31 01:13:02,c97cf355-964c-4481-8c72-e3e3b7b31399
2,615da99d690c7486296ebdb7,"{'name': 'Massive Attack', 'mbid': '10adbe5e-a...",Protection (The Eno Mix),"[trip-hop, chillout, Massive Attack, electroni...",False,0,53855,2007-07-31 01:16:23,03c6743e-a300-49ec-bf9f-e66505c95bdd
3,615da99b690c7486296ebdb6,"{'name': 'Apollo 440', 'mbid': '1ff10dff-7ac7-...",The Machine in the Ghost,"[electronic, electronic rock, chillout, ambien...",False,0,67939,2007-07-31 01:16:23,None
4,615da998690c7486296ebdb5,"{'name': 'Chicane', 'mbid': 'c3480e79-a8d9-44f...",Early,"[chillout, ambient, trance, electronic, Chican...",False,0,209607,2007-07-31 12:44:18,f5f63906-1c43-4602-8fe2-0648923759ed
...,...,...,...,...,...,...,...,...,...
88193,615ac6fdbbaca89fcc4b993c,"{'name': 'Congorock', 'mbid': '763666a7-f40c-4...",Runark - Jokers Of The Scene Remix,[],False,0,49,2021-10-02 20:36:50,None
88194,615ac6fabbaca89fcc4b993b,"{'name': 'Daft Punk', 'mbid': '056e4f3e-d505-4...",Human After All / Together / One More Time / M...,[2007],False,0,70848,2021-10-02 20:43:55,40a954e5-fa4b-46e4-aa8f-d154e00d0635
88195,615ac6f8bbaca89fcc4b993a,"{'name': 'Paris Angels', 'mbid': '6978ae59-f96...",Perfume (Loved Up),"[dance, Manchester, underrated, madchester, ea...",False,0,21659,2021-10-02 20:53:54,e73dc5de-ef91-4d91-9b6a-fafe58ad2c07
88196,615ac6f5bbaca89fcc4b9939,"{'name': 'Closer Musik', 'mbid': '5ad42a9e-838...",You Don´t Know Me,[title is declarative],False,0,1133,2021-10-02 20:59:52,None


Get top-10 tags by year

In [50]:
tags = plays[["tags", "playback_utc_date"]]

def most_common_tag(series):
    return Counter(chain.from_iterable(series)).most_common()

most_common_tags_by_year = tags.resample(rule='y', on='playback_utc_date')["tags"].apply(most_common_tag2).reset_index().set_index("playback_utc_date")
most_common_tags_by_year
for moment in most_common_tags_by_year.index:
    print("-------", moment.year, "-----" )
    for tags in most_common_tags_by_year.loc[moment]:
        for t, count in tags:
            if count > 200:
                print("-", t, count) 
            

------- 2007 -----
- electronic 753
- electronica 508
- dance 403
- chillout 330
- trance 312
- techno 275
- alternative 236
- rock 212
- House 206
- ambient 205
------- 2008 -----
- electronic 4361
- electronica 3102
- chillout 2421
- dance 1839
- ambient 1611
- trip-hop 1333
- downtempo 1330
- electro 1222
- techno 1189
- House 1108
- chill 1055
- trance 993
- female vocalists 971
- alternative 886
- lounge 818
- pop 780
- indie 650
- rock 630
- british 623
- minimal 533
- instrumental 522
- 90s 359
- trip hop 340
- beautiful 329
- jazz 319
- progressive trance 309
- Soundtrack 299
- minimal techno 288
- big beat 271
- Progressive House 257
- Moby 253
- alternative rock 252
- french 247
- breakbeat 236
- Electroclash 230
- 00s 217
- electropop 216
- Mellow 214
- vocal trance 214
- club 213
- nu jazz 213
- experimental 203
- Dreamy 202
------- 2009 -----
- electronic 4130
- electronica 2819
- chillout 1795
- electro 1700
- dance 1602
- ambient 1449
- downtempo 1147
- alternative 973
-

In [32]:



    
most_common_tags_by_month = tags.resample(rule='M', on='playback_utc_date')["tags"].apply(most_common_tag2).reset_index()
most_common_tags_by_month

,playback_utc_date,tags
0,2007-07-31,"[(electronic, 26), (electronica, 19), (chillou..."
1,2007-08-31,"[(electronic, 224), (electronica, 154), (dance..."
2,2007-09-30,"[(electronic, 93), (electronica, 57), (dance, ..."
3,2007-10-31,"[(electronic, 78), (electronica, 50), (dance, ..."
4,2007-11-30,"[(electronic, 66), (rock, 38), (alternative, 3..."
...,...,...
167,2021-06-30,"[(electronic, 65), (indie, 36), (dance, 27), (..."
168,2021-07-31,"[(electronic, 57), (chillout, 28), (dance, 26)..."
169,2021-08-31,"[(electronic, 83), (House, 47), (electronica, ..."
170,2021-09-30,"[(dance, 52), (electronic, 49), (pop, 42), (Ho..."


,playback_utc_date,tags
0,2007-07-31,"[(electronic, 26), (electronica, 19), (chillou..."
1,2007-08-31,"[(electronic, 224), (electronica, 154), (dance..."
2,2007-09-30,"[(electronic, 93), (electronica, 57), (dance, ..."
3,2007-10-31,"[(electronic, 78), (electronica, 50), (dance, ..."
4,2007-11-30,"[(electronic, 66), (rock, 38), (alternative, 3..."
...,...,...
167,2021-06-30,"[(electronic, 65), (indie, 36), (dance, 27), (..."
168,2021-07-31,"[(electronic, 57), (chillout, 28), (dance, 26)..."
169,2021-08-31,"[(electronic, 83), (House, 47), (electronica, ..."
170,2021-09-30,"[(dance, 52), (electronic, 49), (pop, 42), (Ho..."


First, load data into a data frame

Extract top tags

In [41]:
plays["tags"]

0                                                      []
1       [trip-hop, chillout, electronic, downtempo, tr...
2       [downtempo, chillout, trip-hop, electronic, tr...
3       [chillout, best of chill, downtempo, Best Of D...
4       [Soundtrack, Hip-Hop, 90s, space jam, 80's, mo...
                              ...                        
6384                                                   []
6385    [chillout, lounge, downtempo, ambient, chill, ...
6386    [funky house, Disco, deep house, essentials, n...
6387    [lounge, chillout, downtempo, ambient, electro...
6388                              [bujaki, bajki z marsa]
Name: tags, Length: 6389, dtype: object